In [43]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

import os

import numpy as np
import sep
import matplotlib.pyplot as plt
%matplotlib auto

from astropy.io import fits

import gphelper as gp
import ImageTools as it
import DataTools as dt

Using matplotlib backend: Qt5Agg


In [2]:
def make_nonnegative(img):
    epsilon = 0 if len(img[img<0])==0 else img[img>0].min()
    return img + abs(img.min()) + epsilon

def get_random_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources

In [72]:
def convergence_update(re, 
                       is_converged, 
                       int_at_re, 
                       int_at_limit,
                       rs, 
                       convergence_tracking,
                       count, 
                       max_attempts):
    tolerance = 1e-3
    ratio = (int_at_re/int_at_limit-0.5)

    current_re_idx = np.nonzero(rs==re)[0][0]
    if (ratio > tolerance):
        if current_re_idx==0:
            raise Exception('Trying to go negative')
        re = rs[current_re_idx-1]
    elif (ratio < -tolerance):
        re = rs[current_re_idx+1]
    else:
        is_converged = True
        convergence_tracking['converged'].append(re)
        
    #print(f'New re:{re} from ratio:{int_at_re}/{int_at_limit}-0.5={ratio}')
    
    check_stop_condition(count, max_attempts, re)
    
    
    return re, is_converged

def check_stop_condition(count, max_attempts, re):
    stop = False
    
    int_limit = 0
    # we keep bouncing between values take the closest one
    if count==max_attempts:
        raise Exception('max_attempts')
        stop = True

        int_limit = closest_r(rs, 5*re)
        re_idx = ((np.cumsum(fs[rs<int_limit])/fs[rs<int_limit].sum())-0.5).argmin()
        re = rs[re_idx]
        convergence_tracking['max_attempts'].append(re)
    
    # the source has outgrown our image
    elif 5*re >= 42:
        raise Exception('overgrown')
        stop = True
        
        int_limit = closest_r(rs, 42)
        re_idx = np.square((np.cumsum(fs[rs<int_limit])/fs[rs<int_limit].sum())-0.5).argmin()
        re = rs[re_idx]
        convergence_tracking['overgrown'].append(re)
        
    return re, stop, int_limit

def segmap_too_small(src_map):
    area = src_map.sum()
    r = np.sqrt(area/np.pi)
    return r<=15

def closest_r(rs, r):
    r_idx = np.square(rs-r).argmin()
    return rs[r_idx]

def in_segmap_integration(img, rs, src_map):
    rs = rs[src_map].flatten()
    fs = img[src_map].flatten()

    sorted_rs = np.argsort(rs)
    rs = rs[sorted_rs]
    fs = fs[sorted_rs]
    fs[fs<0] = 0

    fs_int = np.cumsum(fs)

    re_idx = np.argmin(np.square(fs_int/fs.sum()-0.5))
    re = rs[re_idx]
    ie = fs[re_idx]

    fs /= ie
    rs /= re

    fs_int = np.cumsum(fs) / fs.sum()
    
    return rs, fs_int, re

    
    
convergence_tracking = {a:[] for a in ['converged', 'too_small', 'overgrown', 'max_attempts']}
spheroids = get_random_spheroids(num=-1)
found_res, rs_re, fs_ie = [], [], []

for img, segmap, img_id in spheroids:
    src_map = segmap==img_id
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(array_sep_likes, mask=src_map,bw=10,bh=10)
    img_sub = make_nonnegative(img-bkg)

    cx, cy = it.img_center(img, src_map)
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    _rs = np.sqrt(np.square(cx-xs) + np.square(cy-ys))
    
    if segmap_too_small(src_map):
        rs, fs_int, re = in_segmap_integration(img_sub, _rs, src_map)
        convergence_tracking['too_small'].append(re)
    else:
        # find using iterative method
        rs = _rs.flatten()
        fs = img_sub.flatten()

        sorted_rs = np.argsort(rs)
        rs = rs[sorted_rs]
        fs = fs[sorted_rs]

        fs_int = np.cumsum(fs)

        re = closest_r(rs, rs.max()*0.1)
        is_converged = False
        max_attempts = 1000
        count = 0

        while not is_converged:
            int_limit = closest_r(rs, 5*re)
            int_at_re = fs_int[rs==re]
            int_at_limit = fs_int[rs==int_limit]
            
            try:
                re, is_converged = convergence_update(re, 
                                                      is_converged,
                                                      int_at_re,
                                                      int_at_limit,
                                                      rs, 
                                                      convergence_tracking,
                                                      count,
                                                      max_attempts)
            except Exception as e:
                msg = str(e)
                
                # not going to converge, use segmap
                if msg=='overgrown':
                    rs, fs_int, re = in_segmap_integration(img_sub, _rs, src_map)
                    convergence_tracking[msg].append(re)
                    is_converged = True
                    
                # we're bouncing between pixels pick the closest one
                elif msg=='max_attempts':
                    int_mask = rs<=int_limit
                    fs_int = np.cumsum(fs[int_mask])/fs[int_mask].sum()
                    re_idx = np.square(fs_int-0.5).argmin()
                    re = rs[re_idx]
                    convergence_tracking[msg].append(re)
                    is_converged = True
                else:
                    raise(msg)

            count += 1
            
        ie = fs[rs==re].mean()
        
        fs = fs[rs<=int_limit]
        rs = rs[rs<=int_limit]
        
        rs /= re
        fs /= ie
        
        fs_int = np.cumsum(fs) / fs.sum()
        
    rs_re.append(rs)
    fs_ie.append(fs_int)
        
    found_res.append(re)
        
    if len(spheroids)<5:
        plt.figure()
        plt.title(img_id)
        plt.plot(rs, fs_int)
        plt.vlines([1], fs_int.min(), 1)
        plt.hlines([0.5], 0, rs.max())
                   
        plt.show()

/home/ryanhausen/anaconda3/envs/sep/lib/python3.6/site-packages/ipykernel/__main__.py:158: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 7056 but corresponding boolean dimension is 931
/home/ryanhausen/anaconda3/envs/sep/lib/python3.6/site-packages/ipykernel/__main__.py:158: RuntimeWarning: Mean of empty slice.
/home/ryanhausen/anaconda3/envs/sep/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/ryanhausen/anaconda3/envs/sep/lib/python3.6/site-packages/ipykernel/__main__.py:160: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 7056 but corresponding boolean dimension is 931
/home/ryanhausen/anaconda3/envs/sep/lib/python3.6/site-packages/ipykernel/__main__.py:158: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 7056 but corres

In [87]:
font_size = 20

plt.figure()
plt.title('$R_e$ Histogram', fontsize=font_size)
plt.xlabel('$R_e$', fontsize=font_size)

for k in convergence_tracking.keys():
    plt.hist(convergence_tracking[k], alpha=0.3, label=f'{k}({len(convergence_tracking[k])})')

plt.legend(fontsize=font_size)
plt.show()

In [90]:
plt.figure()
plt.title('Cumulative Surface Brightness Profile', fontsize=font_size)
plt.xlabel('$R/R_e$', fontsize=font_size)
plt.ylabel("$\sum_r^R I_r$ / $\sum_r^{Rmax} I_r$", fontsize=font_size)

all_rs = []
all_fs = []
for r, f in zip(rs_re, fs_ie):
    all_rs.extend(r.tolist())
    all_fs.extend(f.tolist())
    plt.plot(r, f, '.', color='gray', alpha=0.1, zorder=0)

all_rs = np.array(all_rs)
all_fs = np.array(all_fs)
sorted_rs = np.argsort(all_rs)
all_rs = all_rs[sorted_rs]
all_fs = all_fs[sorted_rs]

bins = np.linspace(all_rs.min()*0.1, all_rs.max()*1.1, 50)
bin_vals = {b:[] for b in bins}
for r,f in zip(all_rs, all_fs):
    bin_idx = bins-r
    bin_idx[bin_idx < 0] = 10
    bin_idx = bin_idx.argmin()
    bin_vals[bins[bin_idx]].append(f)

ms, stds = [], []
for b in bins:
    ms.append(np.nanmean(bin_vals[b]))
    stds.append(np.nanstd(bin_vals[b]))
#bins = [(b2+b1)/2 for b1,b2 in zip(bins[:-1], bins[1:])]
bins = bins - np.diff(bins).mean()/2
#print(ms)
#print(stds)
plt.errorbar(bins, ms, yerr=stds, fmt='o', color='r', zorder=10)
    
plt.show()

/home/ryanhausen/anaconda3/envs/sep/lib/python3.6/site-packages/ipykernel/__main__.py:29: RuntimeWarning: Mean of empty slice
/home/ryanhausen/anaconda3/envs/sep/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
